In [2]:
import pandas as pd
import json
import time
from google_currency import convert 
import warnings
warnings.filterwarnings('ignore')


In [10]:
df=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/country_currency.csv')
df.head()

,Country,last_value,Currency,Code
0,aruba,1.350690,Aruban Florin,AWG
1,afghanistan,17.914483,Afghani,AFN
2,angola,146.167168,Kwanza,AOA
3,albania,41.342391,Lek,ALL
4,united arab emirates,2.258503,UAE Dirham,AED


# PPP to dollar

https://pypi.org/project/google-currency/

!pip install google-currency

In [8]:
def ppp_to_dollar(Code, last_value):
        conversion=convert(Code, 'usd', last_value)
        result = json.loads(conversion)
        return result['amount']

In [4]:
ppp_to_dollar('EUR', 8229.53)

'9789.73'

In [6]:
%%time
apply = df.apply(
         lambda x: ppp_to_dollar(x['Code'], x['last_value']),
         axis=1)
df['ppp_todollar']  = apply
print(df)

Invalid currency codes passed in parameters, original exception message is -> 'BOV'
Invalid currency codes passed in parameters, original exception message is -> 'CHE'
Invalid currency codes passed in parameters, original exception message is -> 'CHW'
Invalid currency codes passed in parameters, original exception message is -> 'COU'
Invalid currency codes passed in parameters, original exception message is -> 'CUC'
Invalid currency codes passed in parameters, original exception message is -> 'ERN'
Invalid currency codes passed in parameters, original exception message is -> 'GHS'


In [ ]:
df["ppp_todollar"] = df.ppp_todollar.astype(float)

In [11]:
# adjustment 
adjustment=1/(df.ppp_todollar[df.Country == 'spain'])
adjustment

AttributeError: 'DataFrame' object has no attribute 'ppp_todollar'

In [ ]:
def ppp_to_spain(ppp_to_dollar):
    return round(ppp_to_dollar*adjustment, 2)

In [ ]:
%%time
apply = df.apply(
         lambda x: ppp_to_spain(x['ppp_todollar']),
         axis=1)
df['ppp_spain']  = apply

CPU times: user 24 ms, sys: 1.79 ms, total: 25.8 ms
Wall time: 26.4 ms


In [ ]:
# Fix Germany
df.ppp_todollar[df.Country == 'germany']=ppp_to_dollar('EUR', 0.733679)
df.ppp_spain[df.Country == 'germany']=ppp_to_spain(0.87)

<ipython-input-9-16c42b70b067>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ppp_todollar[df.Country == 'germany']=ppp_to_dollar('EUR', 0.733679)
<ipython-input-9-16c42b70b067>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ppp_spain[df.Country == 'germany']=ppp_to_spain(0.87)


In [ ]:
df[df.Country == 'germany']

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
44,germany,0.744679,Euro,EUR,0.87,1.17


In [ ]:
df.dtypes

Country          object
last_value      float64
Currency         object
Code             object
ppp_todollar     object
ppp_spain       float64
dtype: object

In [ ]:
df["ppp_spain"] = df.ppp_spain.astype(float)

In [ ]:
df[df.ppp_todollar == 0.00]

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
23,bolivia,2.694105,Mvdol,BOV,0,0.0
32,switzerland,1.143836,WIR Euro,CHE,0,0.0
34,switzerland,1.143836,WIR Franc,CHW,0,0.0
40,colombia,1351.775746,Unidad de Valor Real,COU,0,0.0
44,cuba,0.321939,Peso Convertible,CUC,0,0.0
56,eritrea,5.469904,Nakfa,ERN,0,0.0
66,ghana,2.032734,Ghana Cedi,GHS,0,0.0
107,mexico,9.505202,Mexican Unidad de Inversion (UDI),MXV,0,0.0
113,mongolia,898.732634,Tugrik,MNT,0,0.0
114,mozambique,23.547195,Mozambique Metical,MZN,0,0.0


In [ ]:
df.drop(df[df['ppp_todollar'] == 0.0].index, inplace = True)

# Adjustment function

In [ ]:
def price_calculator(product_price, country):
    find_ppp=df.ppp_spain[df.Country == country.lower()]
    return product_price * find_ppp

In [ ]:
price_calculator(60, 'colombia')

39    28.2
Name: ppp_spain, dtype: float64

In [ ]:
conversion=convert('usd', 'eur', 60)
result = json.loads(conversion)
result['amount']

'50.45'

In [ ]:
df.to_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx', index=False)

In [ ]:
df=pd.read_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx')
df

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
0,aruba,1.350690,Aruban Florin,AWG,0.75,0.98
1,afghanistan,17.914483,Afghani,AFN,0.23,0.30
2,angola,146.167168,Kwanza,AOA,0.23,0.30
3,albania,41.342391,Lek,ALL,0.40,0.52
4,united arab emirates,2.258503,UAE Dirham,AED,0.61,0.80
...,...,...,...,...,...,...
160,uruguay,25.351299,Peso Uruguayo,UYU,0.58,0.76
161,united states,1.000000,US Dollar,USD,1.00,1.31
162,uzbekistan,2085.548458,Uzbekistan Sum,UZS,0.20,0.26
163,south africa,6.652566,Rand,ZAR,0.46,0.60


In [ ]:
df[df.Country == 'spain']

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
51,spain,0.624484,Euro,EUR,0.74,1.0


# Forecast transformation

In [3]:
forecast_currency=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/forecast_currency.csv')

In [4]:
forecast_currency.head()

,1990-01-01 00:00:00,1991-01-01 00:00:00,1992-01-01 00:00:00,1993-01-01 00:00:00,1994-01-01 00:00:00,1995-01-01 00:00:00,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,...,2018-01-01 00:00:00,2019-01-01 00:00:00,2020-01-01 00:00:00,2021-01-01 00:00:00,2022-01-01 00:00:00,2023-01-01 00:00:00,Country,Entity,Currency,Code
0,9.451314e-01,9.655218e-01,9.798110e-01,1.007829,1.048954,1.062042,1.077156,1.098135,1.157248,1.165873,...,1.409705,1.424593,1.439480,1.455043,1.452454,1.459687,aruba,aruba,Aruban Florin,AWG
1,9.190297e+00,9.190297e+00,9.190297e+00,9.190297,9.190297,9.190297,9.190297,9.190297,9.190297,9.190297,...,17.144309,17.914483,18.927627,19.393423,19.372971,19.287213,afghanistan,afghanistan,Afghani,AFN
2,1.058393e-08,2.112144e-08,1.190553e-07,0.000001,0.000026,0.000497,0.023940,0.045998,0.063389,0.410850,...,117.116076,146.167168,174.713839,248.139497,225.226384,200.785136,angola,angola,Kwanza,AOA
3,2.338549e+00,3.065402e+00,9.979887e+00,21.998489,29.258513,31.477205,35.424557,38.738167,40.720081,41.101613,...,42.125796,41.342391,41.497269,42.474337,42.308328,42.015124,albania,albania,Lek,ALL
4,1.167017e+00,1.137996e+00,1.132743e+00,1.120668,1.094358,1.113766,1.156881,1.126440,1.066513,1.140101,...,2.346952,2.258503,2.389256,2.381453,2.381787,2.380031,united arab emirates,united arab emirates,UAE Dirham,AED


In [ ]:
#df_numeric=forecast_currency.select_dtypes(include='float64')

In [5]:
def update_fc(value, code):
  dollar = ppp_to_dollar(code, value)
  adapted = ppp_to_spain(float(dollar))
  return adapted

In [7]:
update_fc(1.007829, 'AWG')

NameError: name 'ppp_to_dollar' is not defined

In [ ]:
%%time
#for i in forecast_currency:
    #forecast_currency[i]=forecast_currency.apply(
        #lambda x: update_fc(x[i], x['Code']),
        #axis=1)

KeyError: 'Code'

In [ ]:
update_fc(forecast_currency['1990-01-01 00:00:00'][0])

0.29
51    0.39
Name: ppp_todollar, dtype: float64


In [ ]:
forecast_currency

,51
0,0.30
1,0.16
2,0.00
3,0.03
4,0.43
...,...
180,0.00
181,0.00
182,0.12
183,0.00


In [ ]:
forecast_currency

,1990-01-01 00:00:00,1991-01-01 00:00:00,1992-01-01 00:00:00,1993-01-01 00:00:00,1994-01-01 00:00:00,1995-01-01 00:00:00,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,...,2018-01-01 00:00:00,2019-01-01 00:00:00,2020-01-01 00:00:00,2021-01-01 00:00:00,2022-01-01 00:00:00,2023-01-01 00:00:00,Country,Entity,Currency,Code
0,3.900000e-01,9.655218e-01,9.798110e-01,1.007829,1.048954,1.062042,1.077156,1.098135,1.157248,1.165873,...,1.409705,1.424593,1.439480,1.455043,1.452454,1.459687,aruba,aruba,Aruban Florin,AWG
1,9.190297e+00,9.190297e+00,9.190297e+00,9.190297,9.190297,9.190297,9.190297,9.190297,9.190297,9.190297,...,17.144309,17.914483,18.927627,19.393423,19.372971,19.287213,afghanistan,afghanistan,Afghani,AFN
2,1.058393e-08,2.112144e-08,1.190553e-07,0.000001,0.000026,0.000497,0.023940,0.045998,0.063389,0.410850,...,117.116076,146.167168,174.713839,248.139497,225.226384,200.785136,angola,angola,Kwanza,AOA
3,2.338549e+00,3.065402e+00,9.979887e+00,21.998489,29.258513,31.477205,35.424557,38.738167,40.720081,41.101613,...,42.125796,41.342391,41.497269,42.474337,42.308328,42.015124,albania,albania,Lek,ALL
4,1.167017e+00,1.137996e+00,1.132743e+00,1.120668,1.094358,1.113766,1.156881,1.126440,1.066513,1.140101,...,2.346952,2.258503,2.389256,2.381453,2.381787,2.380031,united arab emirates,united arab emirates,UAE Dirham,AED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,7.645164e+01,8.154396e+01,8.194840e+01,81.702942,81.842870,81.550365,81.753910,82.921362,88.687148,90.116402,...,107.438000,108.104856,109.555167,110.683458,110.519054,110.194778,vanuatu,vanuatu,Vatu,VUV
181,7.440918e-01,7.637383e-01,8.106851e-01,0.797107,1.312921,1.196255,1.228667,1.350804,1.381368,1.395503,...,1.658307,1.662674,1.878549,1.844421,1.850386,1.834103,samoa,samoa,Tala,WST
182,1.265262e+00,1.415433e+00,1.585498e+00,1.749861,1.877083,2.026311,2.147189,2.279385,2.429536,2.563279,...,6.519588,6.652566,6.609416,6.651762,6.638367,6.624265,south africa,south africa,Rand,ZAR
183,9.556399e-03,1.780860e-02,4.623424e-02,0.110046,0.194742,0.249130,0.304221,0.375034,0.433561,0.503995,...,4.396237,4.641657,4.098580,3.315622,3.515256,3.770987,zambia,zambia,Zambian Kwacha,ZMW


In [ ]:
def update_fc(value, code)